# MATH 405/607 

# Chaos In The Context Of Dynamical Systems

Callum Hepworth, Sandy Luo, Jed Yeo


## Content

1) Introduction to Chaos Theory
2) Strange Attractor & Lorenz Problem
3) Popular Examples of Chaos
4) Studying Chaos Numerically (ODEs)

## Introduction

- Chaos theory 
    - "Study of random or unpredictable behaviour in systems governed by deterministic laws" [1]
    - Example: Pinball machine
- Applications
    - Turbulent flow
    - Chemical reactions
    - Planetary motion

- Dynamical systems
    - Highly sensitive to initial conditions
    - i.e, adjusting initial conditions by some amount leads to interesting changes in the solution
    - Behavior of dynamical chaotic systems appears random
    - This is not the case - these systems are deterministic (future dynamics are fully defined by initial conditions)

## Lorenz Heat Problem

- Chaos first observed by Edward Lorenz in 1963 
- The problem: atmospheric convection (three ODEs):

$$\frac{dx}{dt} = \sigma(y-x), \frac{dy}{dt}=x(\rho-z)-y, \frac{dz}{dt}=xy-\beta z $$


- These three equations relate the properties of a 2D fluid layer uniformly warmed from below and cooled from above
- $x$ is proportional to rate of convection, $y$ to horizontal temperature gradient, $z$ to the vertical gradient
- Constants depend on physical properties of the system



In [1]:
# We take σ = 10, ρ = 28, β = 8/3
include("math405.jl")
using OrdinaryDiffEq, Plots
function lorenz(du,u,p,t)
    du[1] = 10.0*(u[2]-u[1])
    du[2] = u[1]*(28.0-u[3]) - u[2]
    du[3] = u[1]*u[2] - (8/3)*u[3]
end

lorenz (generic function with 1 method)

In [ ]:
u0 = [-15.0;-15.0;20.0]
tspan = (0.0,100.0)
prob = ODEProblem(lorenz,u0,tspan)
sol = solve(prob, Tsit5())
plot(sol,vars=(1,2,3))

In [ ]:
u1 = [-15.0;-15.0;20.0001]
prob = ODEProblem(lorenz,u1,tspan)
sol = solve(prob, Tsit5())
plot(sol,vars=(1,2,3))

- These butterfly-like shapes reveal the structure of the "stranger attractor" for the Lorenz problem
- A mere adjustment of $1\times10^{-4}$ in the third initial condition yields a vastly different solution space

- Lorenz's choices for $\beta = \frac{8}{3}$, $\sigma = 10$, and $\rho = 28$ were intentional.

- With these parameters almost all IC's tend toward an invariant set of states as the system evolves numerically.

- For a general ODE this set of states is called an $\textbf{attractor}$, and if these states form a fractal they are called a $\textbf{strange attractor}$.

    - Because it's a notable example, the strange attractor that emerges from the Lorenz system is dubbed the $\textbf{Lorenz attractor}$.

- While the Lorenz attractor is a notable example, these behaviours emerge from a variety of dynamical systems.

- Most often, but not always, strange attractors emerge from chaotic systems.

- As Jed mentioned in the introduction, a necessary condition of a chaotic system is that the behaviour of the system is heavily dependent on the choice of initial conditions.

## Sensitivity to Initial Conditions

- We can visualize this initial condition dependence by taking a dynamical system that is known to be chaotic, i.e. the Lorenz system with $\beta = \frac{8}{3}$, $\sigma = 10$, and $\rho = 28$, and perturb $z_0$.

In [ ]:
u1 = [-15.0;-15.0;20.0]
prob = ODEProblem(lorenz,u1,tspan)
sol_unperturbed = solve(prob, Tsit5())

u1 = [-15.0;-15.0;20.00001]
prob = ODEProblem(lorenz,u1,tspan)
sol_perturbed = solve(prob, Tsit5())

plot(sol_unperturbed,vars=(1), color=1, xlims=(0, 20), ylabel="x(t)", title="Incremental x(t) vs. Time for the Lorenz System")
plot!(sol_perturbed,vars=(1), color=2, xlims=(0, 20))

In [ ]:
plot(sol_unperturbed,vars=(2), color=1, xlims=(0, 20), ylabel="y(t)", title="Incremental y(t) vs. Time for the Lorenz System")
plot!(sol_perturbed,vars=(2), color=2, xlims=(0, 20))

In [ ]:
plot(sol_unperturbed,vars=(3), color=1, xlims=(0, 20), ylabel="z(t)", title="Incremental z(t) vs. Time for the Lorenz System")
plot!(sol_perturbed,vars=(3), color=2, xlims=(0, 20))

- Despite having $z_0$ differ by one part in $10^6$, the systems diverge wildly after the $\approx 13^{th}$ timestep.
- To briefly quantify this behaviour, we note that an initial state difference of $10^{-5}$ in $z_0$ has grown by six orders of magnitude over $15$ time steps.

### Can we quantify this rate of divergence?

- To quantify trends in the sensitivity of chaotic dynamical systems to initial conditions, it is useful to first visualize the rate of divergence over time.

- We can do so by plotting the $l$-2 norm of the difference between the perturbed and unperturbed states as a function of time. 
    
    - This has been done for the Lorenz system below.


In [ ]:
TT = 1:1:500
err = @. sqrt((sol_perturbed[1, TT]-sol_unperturbed[1, TT])^2+(sol_perturbed[2, TT]-sol_unperturbed[2, TT])^2+(sol_perturbed[3, TT]-sol_unperturbed[3, TT])^2)

TT = (1:1:500) * (40 / 700)
plot(TT,err, yaxis=:log, xlabel="t", ylabel=L"|| \hat{y}(t) - y(t) ||_{2}", title=L"l" * "-2 Norm of Perturbed and Unperturbed State Difference vs. Time", label="", size=(800,400))

- Applying this visualization reveals two distinct regions in the growth of the perturbations in our initially perturbed and unperturbed systems.

- When T > 15, the $l$-2 error appears to reach an equilibrium and stops growing.

- When T < 15, the $l$-2 error appears to grow exponentially. 

- In general, the rate of growth in this initial exponential regime for a particular ODE is called it's $\textbf{Lyapunov exponent}$. 

- This exponent is defined as $\approx 0.9052$ for the Lorenz system. 

- Plotting an exponential function with this Lyapunov exponent looks as follows.

In [ ]:
TT = 1:1:500
err = @. sqrt((sol_perturbed[1, TT]-sol_unperturbed[1, TT])^2+(sol_perturbed[2, TT]-sol_unperturbed[2, TT])^2+(sol_perturbed[3, TT]-sol_unperturbed[3, TT])^2)

TT = (1:1:500) * (40 / 700)
plot(TT,err, yaxis=:log, xlabel="t", ylabel=L"|| \hat{y}(t) - y(t) ||_{2}", title=L"l" * "-2 Norm of Perturbed and Unperturbed State Difference vs. Time",legend=:bottomright, label="perturbation norm", size=(800,400))

TT = (1:1:300) * (40 / 700)
plot!(TT, 0.0001 *exp.(TT * 0.9052), label=L"\lambda = 0.9052")

- We can say that perturbations in Lorenz systems typically grow at a rate of:

\begin{align*}
    \| \hat{y}(t) - y(t) \|_2 &\approx Ce^{0.9052t}
\end{align*}

In general, the formal definition of a Lyapunov exponent is given by:

\begin{align*}
    \lambda &= \underset{t \rightarrow \infty}{lim} \underset{\| \delta y(0) \| \rightarrow 0}{limsup} t^{-1} log \frac{\| \delta y(t) \|}{\| \delta y(0) \|},
\end{align*}

where:

\begin{align*}
    \delta y(t) &= \hat{y}(t) - y(t)
\end{align*}

## Popular Examples of Chaos

i love examples

- Rossler equations
- Chaotic nonlinear forced operator

In [ ]:
example = 0

## Studying Chaos Numerically

- The process of solving chaotic problems numerically is no different from that of a regular ODE system
- e.g. the Lorenz problem is simply a system of 3 IVPs

In [ ]:
study = 6

## Influence of Round-Off Errors

- When dealing with systems that are hypersensitive to initial perturbations, it is natural to want to investigate the impacts that round-off errors, an unfortunate byproduct of numerical computing, can have on the reliability of their simulation. 

- We will now examine these complications in the case of our pedagogical example of the Lorenz system, generalizing the example that we provided previously.

- In our previous example, we varied the step size in our numerical solver of the Lorenz system with parameters designed to elicit chaotic behaviour and observed a scattered distribution of end points. 

- In order to clearly observe the effects of round-off errors on the evolution of our system, it would be ideal if we had a limited number of well defined end points, between which we might observe oscillation as a function of step size.

- Conveniently the Lorenz system has such a property when $\rho$ is chosen as $1 < \rho < 24.74$, with $\beta = \frac{8}{3}$ and $\sigma = 10$. It is well known that with these initial conditions, fixed point attractors emerge at:

\begin{gather*}
x = \sqrt{\beta(\rho - 1)}, y = \sqrt{\beta(\rho-1)}, z = \rho - 1, \\
and \\
x = -\sqrt{\beta(\rho - 1)}, y = -\sqrt{\beta(\rho-1)}, z = \rho - 1
\end{gather*}

- To leverage this regime we will select $\rho = 23$, with $\beta$ and $\sigma$ from before.

- It has been shown that the 4th-order Runge-Kutta method with double precision exhibits a particular sensitivity to the choice of step-size $h$ [3].

- Despite efforts to reduce $h$ and thus shrink truncation errors, final values over identical time horizons consistently fluctate between our two expected fixed points. 

In [ ]:
# using ImageView
# img = load(“numerical-chaos/chaotic_fluctuation.png”)
# imshow(img)

display("image/png", read("chaotic_fluctuation.png"))

- Even when directly isolating round-off errors as the primary source of uncertainty by taking the truncation error to the $M$th-order, with $M > 30$, fluctuations in final values emerge in compute platforms that leverage multiple concurrent processes. 

- In order to avoid the uncertainty introduced by numerically solving the Lorenz equation, and indeed chaotic systems in general, it is crucial that both truncation and round-off errors be decreased simultaneously. 

- In fact, using the Mth-order Taylor series method with $M \geq 130$ with 512-digit precision, numerical agreement can be achieved over the entire time interval even when accounting for varying numbers of computational processes and results obtained across different machines.

- This relationship is illustrated clearly in the following plot. In this case the initial condition $(5,5,10)$ was was selected with $\rho = 23$ and $\beta$ and $\sigma$ from the Lorenz system. 

In [ ]:
display("image/png", read("stability.png"))

- It is crucial to once again consider that this result was obtained on the Lorenz system with $\textit{non-chaotic}$ initial conditions.

- To eliminate troublesome round-off and truncation errors in this regime requires extreme levels of precision.

- In one instance researchers required the use of a $3500$th-order Taylor expansion method with $4180$-point precision to achieve a convergent numerical solution to the chaotic Lorenz equation over a large time horizon $([0, 10000])$.

- It is clear that numerically accurate simulations of chaotic dynamical solutions are particularly difficult to develop. We are eager to explore recent advances in addressing this issue more deeply.

Reference:

[3] : https://www.researchgate.net/publication/318488232_Influence_of_round-off_errors_on_the_reliability_of_numerical_simulations_of_chaotic_dynamic_systems